In [1]:
import openmm
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.interchange.components.interchange import Interchange
from openff.interchange.components.potentials import Potential
from openff.interchange.models import PotentialKey, TopologyKey

In [2]:
propanol = Molecule.from_smiles("CCCO")
propanol.generate_conformers(n_conformers=1)
topology = propanol.to_topology()

sage = ForceField("openff-2.0.0.offxml")

In [3]:
interchange = Interchange.from_smirnoff(sage, topology)

original_openmm_system = interchange.to_openmm()

In [4]:
interchange.handlers["Bonds"].slot_map

{TopologyKey(atom_indices=(0, 1), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#6X4:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(0, 4), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(0, 5), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(0, 6), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(1, 2), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#6X4:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(1, 7), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(1, 8), mult=None, bond_order=None): Pot

In [5]:
for topology_bond in topology.topology_bonds:
    if all(atom.atomic_number == 6 for atom in topology_bond.atoms):
        atom_indices = tuple(atom.topology_atom_index for atom in topology_bond.atoms)
        top_key = TopologyKey(atom_indices=atom_indices)
        pot_key = interchange.handlers["Bonds"].slot_map[top_key]
        print(atom_indices, pot_key.__repr__())

(0, 1) PotentialKey(id='[#6X4:1]-[#6X4:2]', mult=None, associated_handler='Bonds', bond_order=None)
(1, 2) PotentialKey(id='[#6X4:1]-[#6X4:2]', mult=None, associated_handler='Bonds', bond_order=None)


Notice that the `PotentialKey` associated with each of the C-C bonds - atom indices (0, 1) and (1, 2) - is the same, in this case associated with SMIRKS pattern `'[#6X4:1]-[#6X4:2]'`. This means the same parameters have been applied to each. For the sake of an example, let's consider splitting these parameters into two types (without re-running SMIRKS/SMARTS-based atom-typing). Let's increase the force constant of the C-C bond nearest the O atom by 5% (atom indices (1, 2)). Here we are ignoring whether or not it is scientifically wise to make such a modification.

This process will involve
1. Creating a new `PotentialKey` to uniquely identify the new parameters
2. Createing a new `Potential` to store the new parameters
3. Updating the bond handler so that the C-C bond we have selected points to new parameters

In [6]:
# First, clone the existing C-C bond PotentialKey
pot_key_mod = PotentialKey(**pot_key.dict())
pot_key_mod.id = "[#6X4:1]-[#6X4:2]_MODIFIED"

In [7]:
# Inspect the original and modified keys
(pot_key, pot_key_mod)

(PotentialKey(id='[#6X4:1]-[#6X4:2]', mult=None, associated_handler='Bonds', bond_order=None),
 PotentialKey(id='[#6X4:1]-[#6X4:2]_MODIFIED', mult=None, associated_handler='Bonds', bond_order=None))

In [8]:
# Look up the existing potential on these bonds and modify the k value by 5%
pot = interchange.handlers["Bonds"].potentials[pot_key]
pot_mod = Potential(**pot.dict())
pot_mod.parameters["k"] *= 1.05

In [9]:
# Inspect the original and modified keys
(pot, pot_mod)

(Potential(parameters={'k': <Quantity(529.242972, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.52190126, 'angstrom')>}, map_key=None),
 Potential(parameters={'k': <Quantity(555.70512, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.52190126, 'angstrom')>}, map_key=None))

In [10]:
# Update the potential mapping to include the new key and potential key
interchange.handlers["Bonds"].potentials.update({pot_key_mod: pot_mod})

Now that we have modified versions of the potential key and potential, the last step is to update the mappings so that they are applied to the bond. We previously decided that we want to modify the bond between atoms 1 and 2.

In [11]:
# Create a topology key representing this bond
top_key = TopologyKey(atom_indices=(1, 2))

In [12]:
# This topology key is already in the handler, as it points to an existing bond
assert top_key in interchange["Bonds"].slot_map

The slot_map is a dictionary mapping topology keys to potential keys, so we can update the mapping of this topology key to point to our new potential key (which points to the new potential we just added).

In [13]:
interchange.handlers["Bonds"].slot_map[top_key] = pot_key_mod

In [14]:
interchange.handlers["Bonds"].potentials[pot_key_mod]

Potential(parameters={'k': <Quantity(555.70512, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.52190126, 'angstrom')>}, map_key=None)

In [15]:
# openff_sys["Electrostatics"].method = "cutoff"
modified_openmm_system = interchange.to_openmm()

In [16]:
# Look at the force constant in the original and modified OpenMM exports
for force in original_openmm_system.getForces():
    if type(force) == openmm.HarmonicBondForce:
        for bond_idx in range(force.getNumBonds()):
            if force.getBondParameters(bond_idx)[:2] == [1, 2]:
                original_k = force.getBondParameters(bond_idx)[3]

# Look at the modified force constant in an OpenMM export
for force in modified_openmm_system.getForces():
    if type(force) == openmm.HarmonicBondForce:
        for bond_idx in range(force.getNumBonds()):
            if force.getBondParameters(bond_idx)[:2] == [1, 2]:
                modified_k = force.getBondParameters(bond_idx)[3]

# Check that the modified k is 5% more than the original k
assert abs(modified_k / original_k - 1.05) < 1e-12